# This notebook describes the training of the datamodel

**Imports**

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.feature_selection import f_regression
import statsmodels.api as sm
from yellowbrick.regressor import ResidualsPlot
import pickle

**Naming dictionary**<br>
This directory is used to easily change the column names of the DataFrames. I made it a dictionary, because I suspected that when I defined the function 'lr_dict' this function would not have access to the variables that are created under "Unpack dictionary..." (see next codeblock). However, the variables seem to be global (?) and not need any further unpacking, thus making creating the dictionary a bit unnecessary.

In [2]:
# Set a naming dictionary, to easily change names
naming = {
    'm_name': "Model_",
    'all_data': 'Data',
    'x_data': 'Independend data',
    'y_data': 'Dependend data',
    'x_train': 'Independend train data',
    'y_train': 'Dependend train data',
    'x_test': 'Independend test data',
    'y_test': 'Dependend test data',
    'y_pred': 'Predicted data',
    'model': 'Model'
}

# Unpack dictionary to use in script
for key, name in naming.items():
        exec(key + '=name')

**Load trainingsets into dictionairy**<br>
Below an overview of all the datasets that (in principle) are available from the pipeline. However, further in the project we might decide to not make all these sets available, but only the sets we decide to use.

The sets we use to create our linear regression models are loaded into an initial dictionary. I choose a dictionary with slices of the DataFrame, instead of a dataframe itself, so I could later easily 'call' the different slices I needed. I feel that for a dataset this size, this would be a viable option. However, if the dataset would be larger, this method might take too much memory.

In [3]:
# Select datasets to be modelled
datasets = [1, 3, *range(20, 29)] # All datasets (except 0)
print(datasets)

# Initialize dictionary and load datasets
initial_dict = {}
path_datasets = "../tform_db/csv/"

for dataset in datasets:
    model_name = m_name + str(dataset)
    file = f"{path_datasets}db_t{str(dataset)}.csv"
    initial_dict[model_name] = {all_data: pd.read_csv(file)}

[1, 3, 20, 21, 22, 23, 24, 25, 26, 27, 28]


*If necessary, check the datasets that are loaded* 

In [4]:
# Set true if you want to inspect the datasets after loading
if False:
    for num, models in enumerate(initial_dict):
        print(num + 1, initial_dict[models]['Data'].columns)

**Predifine function to clear all files in directory**<br>
*Watch out with this function for deleting the wrong files!* 

This function enables to directory 'models' to be as clean as possible (so it contains only the models that are called in this script).

In [5]:
# Remove files in a specific path with a specific extension
def remove_files(path, ext):
    for f in os.listdir(path):
        if ext in f:
            os.remove(os.path.join(path, f))

**Predefine regression function**<br>
I have chosen to 'experiment' with functions to automatically create the slices of the dataset and put them in a dictionary. As said above, this might be inefficient in regard to memory use, however for a dataset this size it is not (yet) a problem. 

Furthermore I experimented with handling all the different datasets at the same time and putting them in the same dictionary. This too can be consideren memory-inefficient, still the dictionary seems to be small enough to be handled fast. 

The function then models (linear regression) the different datasets and saves the result in the same dataset. This model will later be summarized and saved. Linear regression seems applicable for this type of analysis. Other types (K-nearest, decision tree, random forest, ...) were briefly considered but do not seem to bring extra predictive power to the models, as well as I do not (yet) understand the right application of these methods.

In [11]:
# Defining the linear regression model
def lr_dict(initial_dict):
    # Unpack naming dictionary to use in function
    # for key, name in naming.items():
    #     exec(key + '=name')

    for mod in initial_dict:
        # Per model, split (in)dependend data
        initial_dict[mod][x_data] = initial_dict[mod][all_data].iloc[:, 1:]
        initial_dict[mod][y_data] = initial_dict[mod][all_data].iloc[:, :1]

        # Per model, split train-/test-data
        x_train_d, x_test_d, y_train_d, y_test_d = train_test_split(
            initial_dict[mod][x_data], 
            initial_dict[mod][y_data], 
            test_size=0.7, random_state=42)
        
        initial_dict[mod][x_train] = x_train_d
        initial_dict[mod][y_train] = y_train_d
        initial_dict[mod][x_test] = x_test_d
        initial_dict[mod][y_test] = y_test_d
        
        # Fit model
        lr = LinearRegression()
        lr.fit(initial_dict[mod][x_train], 
                  initial_dict[mod][y_train])

        # Predict and save data
        initial_dict[mod][y_pred] = lr.predict(initial_dict[mod][x_test])
        initial_dict[mod][model] = lr
    
    # Return the new dictionary
    return initial_dict
 

**Predefine summary function**<br>
This function takes the dictionary with multiple models and summarized the contents of the models. There are two levels of summaries to be given: extended (includes variable names, coefficients and p-values) and limited (R2 and SME/RSME only). If desired, the summary can be saved for further analysis.

For the p-values I had to use statsmodel, which essentially does the same as scikit-learn. However, for the purpose of studying scikit-learn, I kept using this package for further modelling and analysis.

In [7]:
# Defining the function to summarize the models
def model_summary(models_dict, level ='extended'):
    # Preloading variables/summary
    var = "Variable"
    coef = "Coefficient"
    sig = "P-value"
    col_names = [var, coef, sig]
    
    param = "Parameters"
    metric = "Metrics"
    summary = {}
    
    for mod in models_dict:
        # Per model, get coefficients
        summary[mod] = mod
        df_int = pd.DataFrame({var: "Intercept", coef: models_dict[mod][model].intercept_, sig: 0})
        df_coef = pd.DataFrame(zip(models_dict[mod][x_test].columns, models_dict[mod][model].coef_[0], 
                                   np.zeros(len(models_dict[mod][x_test]))), columns=col_names)
        summary[mod] = {}
        summary[mod][param] = pd.concat([df_int, df_coef], axis=0, ignore_index=True)

        # Per model, get p-values (inefficiently done with statsmodels, but...)
        x_train_sm = models_dict[mod][x_train].copy()
        x_train_sm.insert(0, 'intercept', np.ones(len(x_train_sm)))
        modsm = sm.OLS(models_dict[mod][y_train], x_train_sm.astype(float))
        modsm = modsm.fit()
        summary[mod][param][sig] = modsm.pvalues.values.round(3)

        # Per model, get the regression metrics
        summary[mod][metric] = {}
        summary[mod][metric]['r2'] = metrics.r2_score(models_dict[mod][y_test], 
                                              models_dict[mod][y_pred])
        summary[mod][metric]['mse'] = metrics.mean_squared_error(models_dict[mod][y_test], 
                                         models_dict[mod][y_pred]) 
        summary[mod][metric]['rmse'] = np.sqrt(summary[mod][metric]['mse'])
        summary[mod][metric]['msle'] = metrics.mean_squared_log_error(models_dict[mod][y_test], 
                                                                models_dict[mod][y_pred])
        summary[mod][metric]['mae'] = metrics.mean_absolute_error(models_dict[mod][y_test], 
                                                          models_dict[mod][y_pred]) 
            
        # Per model, print the summary depending on 'extended' or 'limited' level
        if level == 'extended' or level == 'ext':
            # pd.set_option("display.precision", 3)
            print(mod, ":")
            print(summary[mod][param])
            print("\n")
            print('R-squared: ', '%.4f' % round(summary[mod][metric]['r2'], 4), 
                  '; Mean squared error: ', '%.4f' % round(summary[mod][metric]['mse'] ,4),
                  '; Root mean squared error: ', '%.4f' % round(summary[mod][metric]['rmse'] ,4), sep='')
            print('___ \n')
            # pd.reset_option("display.precision")
        elif level == 'limited' or level == 'lim':
            print(mod, ":", 
                  ' R-squared: ', '%.4f' % round(summary[mod][metric]['r2'], 4), 
                  '; Mean squared error: ', '%.4f' % round(summary[mod][metric]['mse'] ,4),
                  '; Root mean squared error: ', '%.4f' % round(summary[mod][metric]['rmse'] ,4), sep='')
        else:
            print("Please use level ""extended"" (""ext"") or ""limited"" (""lim"") or omit a choice.")
    
    # Return summary to be saved if desired
    return summary

### Fitting and predicting
Here we use the function we created before to slice, fit and precict the datasets and models. The result is cast in models_dict.

In [14]:
# Fit models and save in models_dict
models_dict = lr_dict(initial_dict)

11


### Displaying and comparing model outcomes
Here we display the summary of the models. Part of the code can be turned on/off to show the residual plots.

In [9]:
# Show limited/extended summary
model_summary(models_dict, 'lim')
print() # Otherwise prints return statement (= summary)

# If required, show residual plots
if False:
    for mod in models_dict:
        rplot = ResidualsPlot(models_dict[mod][model],
                              title=f"Residuals of {mod}", 
                              size=(400, 300))
        rplot.fit(models_dict[mod][x_train], models_dict[mod][y_train])
        rplot.score(models_dict[mod][x_test], models_dict[mod][y_test])
        rplot.show()

Model_1: R-squared: 0.9809; Mean squared error: 1.1730; Root mean squared error: 1.0831
Model_3: R-squared: 0.9816; Mean squared error: 1.1316; Root mean squared error: 1.0638
Model_20: R-squared: 0.0799; Mean squared error: 56.5622; Root mean squared error: 7.5208
Model_21: R-squared: 0.9734; Mean squared error: 1.6373; Root mean squared error: 1.2796
Model_22: R-squared: 0.9677; Mean squared error: 1.9880; Root mean squared error: 1.4100
Model_23: R-squared: 0.9267; Mean squared error: 4.5047; Root mean squared error: 2.1224
Model_24: R-squared: 0.9790; Mean squared error: 1.2910; Root mean squared error: 1.1362
Model_25: R-squared: 0.9810; Mean squared error: 1.1693; Root mean squared error: 1.0813
Model_26: R-squared: 0.9243; Mean squared error: 4.6553; Root mean squared error: 2.1576
Model_27: R-squared: 0.9237; Mean squared error: 4.6876; Root mean squared error: 2.1651
Model_28: R-squared: 0.9097; Mean squared error: 5.5527; Root mean squared error: 2.3564



**Interpretation**<br>
The higher the R-squared value of a model, the better the model is able to predict the variance of the dependent variable based on the independend variables. The mean (root) mean squared error also indicated the size of the errorterm, where mean squared error is more sensitive for outliers. 

For the final run, I included 11 models. Two (the 'main contestants', models 1 and 3; see below) because of their high R2 and because they are somewhat different from eachother. Models 20 to 28 are included to give alternative options to the end user: for example if some predictors are not available. For these models the note has to be given that their predictive value is lower or much lower, depending on the missing information. 

We see that there are two 'main contestants' for a definite model, namely model_1 (all variables) an model_3 (all variables, but normalized bmi). Model_25 (all variables, normalized bmi and without sugar) seems to be another contender, however this only shows that sugar is not a very important factor. If all models would be considered, also model 8 would show to have a large R2. However, as that model is very similar to model 3, and we prefer model 3 for the larger R2, we excluded model 8. 

The residual plot of the models needs to have normalized distribution. This seems to be the case for most of the models, however models 23, 26 and 27 (possibly 28) seem to be skewed. For now I do not know how to handle this so I will ignore this fact.

### Saving model with pickle
This concludes the modelling and selection of models. We save to models for later use.

In [10]:
# Can be turned off
if True:
    path_models = "./models/"
    extension = ".pickle"

    # Delete all models in directory
    remove_files(path_models, extension)

    # Save all new models
    for mod in models_dict:
        filename = f"{path_models}{mod}{extension}"
        pickle.dump(models_dict[mod][model], open(filename, 'wb'))

**To improve**
- WHAT DO THE P-VALUES MEAN??